In [1]:
import requests
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import concurrent.futures
import pandas as pd
import imgkit
from bs4 import BeautifulSoup
import panel as pn
from panel.interact import interact
from panel import widgets
import base64
import io

# matplotlib inline plots
%matplotlib inline

In [2]:
def display_folded_lc(target_and_tce):
    target = target_and_tce[0]
    tce = target_and_tce[1] if len(target_and_tce) > 1 else 1
    
    tces = json.loads(requests.get(f"https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/{target}/tces/").content.decode('utf-8'))
    # print(tces)
    # sector = tces['TCE'][0].split(':')[0]
    # Given a dictionary that looks like this, {'TCE': ['s0001-s0036:TCE_1', 's0008-s0008:TCE_1', 's0034-s0034:TCE_1', 's0061-s0061:TCE_1', 's0001-s0036:TCE_2', 's0061-s0061:TCE_2']}, keep only the elements that have TCE_{tce}
    tce = f"TCE_{tce}"
    # For each list, if the string ends in tce, keep it
    relevant_tces = [x for x in tces["TCE"] if x.endswith(tce)]
    sector = relevant_tces[0].split(':')[0]
    # print(sector)

    spectra_plot = requests.get(f"https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/{target}/phaseplot/?tce=TCE_1&sector={sector}")
    # display(HTML(str(spectra_plot.content.decode('utf-8'))))
    display(HTML(str(spectra_plot.content.decode('utf-8'))))
    
def run_in_parallel(method, args):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Start a separate thread for each string.
        future_to_string = {executor.submit(method, arg): arg for arg in args}
        
        for future in concurrent.futures.as_completed(future_to_string):
            string = future_to_string[future]
            try:
                # data = future.result()
                future.result()
            except Exception as exc:
                print(f'{string} generated an exception: {exc}')
            else:
                print(f"Object {string} above plotted successfully.")

# strings = ["string1", "string2", "string3"]
# run_in_parallel(strings)

In [3]:
# Load the astep targets log xlsx file as a pandas dataframe
astep_targets_path = "./data/ASTEP_observations_2023.xlsx" # Run locally, do not push to GitHub
astep_targets_df = pd.read_excel(astep_targets_path, sheet_name="Sheet1")

# Load the tfop submitted targets
tfop_targets2021_path = "./data/tfop_submitted2021.xlsx"
tfop_targets2021_df = pd.read_excel(tfop_targets2021_path, sheet_name="Sheet1")

# And 2022
tfop_targets2022_path = "./data/tfop_submitted2022.xlsx"
tfop_targets2022_df = pd.read_excel(tfop_targets2022_path, sheet_name="Sheet1")

# Rename date to date_obs
astep_targets_df.rename(columns={"date": "date_obs"}, inplace=True)
tfop_targets2021_df.rename(columns={"date": "date_obs"}, inplace=True)
tfop_targets2022_df.rename(columns={"date": "date_obs"}, inplace=True)

# Rename name to target_name
astep_targets_df.rename(columns={"name": "target"}, inplace=True)
tfop_targets2021_df.rename(columns={"name": "target"}, inplace=True)
tfop_targets2022_df.rename(columns={"name": "target"}, inplace=True)


# Combine the three dataframes
frames = [astep_targets_df, tfop_targets2021_df, tfop_targets2022_df]
all_targets_df = pd.concat(frames)

# Rename the columns using a dictionary
all_targets_df = all_targets_df.rename(columns={
    'quality \n(0 - not useful/not data \n1 noisy - to be redone with prose\n2 - ok to submit': 'quality',
})

# Print the columns
print(all_targets_df.columns)
all_targets_df



FileNotFoundError: [Errno 2] No such file or directory: './data/ASTEP_observations_2023.xlsx'

In [16]:
# Get the TICs
astep_tics = all_targets_df["tic_name"].tolist()
print(astep_tics)

# Sanitize the TICs and remove those that do not contain a TIC
astep_tics = [str(tic) for tic in astep_tics if str(tic).startswith("TIC")]
print(astep_tics)

# Remove the TIC prefix
astep_tics = [tic.replace("TIC", "") for tic in astep_tics]
print(astep_tics)

# Delete duplicates
astep_tics = list(dict.fromkeys(astep_tics))
print(astep_tics)

# Given astep_tics, create a list of touples of the form (tic, tce), where the dot is the separator, and the tce might be missing
astep_tics_and_tces = [tic.split(".") for tic in astep_tics]
print(astep_tics_and_tces)
# Make them touples; if the touple has only one element, then add a 1 as the second element
astep_tics_and_tces = [(tic_and_tce[0], tic_and_tce[1] if len(tic_and_tce) > 1 else 1) for tic_and_tce in astep_tics_and_tces]
print(astep_tics_and_tces)

# Make both entries be strings, and if tce has two characters, remove the leading zeroes
astep_tics_and_tces = [(tic_and_tce[0], str(int(tic_and_tce[1]))) for tic_and_tce in astep_tics_and_tces]
print(astep_tics_and_tces)

# Remove the tce 14 entry
astep_tics_and_tces = [tic_and_tce for tic_and_tce in astep_tics_and_tces if tic_and_tce[1] != "14"]
print(astep_tics_and_tces)




['TIC86021841.01', 'TIC86021841.01', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889', 'TIC35516889.01', 'TIC35516889.01', 'TIC35516889.01', 'TIC35516889.01', '              ', '              ', 'TIC271169413.01', 'TIC271169413.01', 'TIC381897917.01', 'TIC381897917.01', 'TIC204698337.01', 'TIC204698337.01', 'TIC190271688.01', 'TIC190271688.01', 'TIC133107143.01', 'TIC133107143.01', 'TIC133107143.01', 'TIC133107143.01', 'TIC258917846.01', 'TIC258917846.01', 'TIC54962195.02', 'TIC54962195.02', 'TIC176797879.14', 'TIC176797879.14', 'TIC277128619.01', 'TIC277128619.01', 'TIC277128619.01', 'TIC277128619.01', 'TIC231721006.01', 'TIC231721006.01', 'TIC150151262.02', 'TIC150151262.02', 'TIC342167886.01', 'TIC342167886.01', 'TIC149601126.01', 'TIC149601126.01', 'TIC260647166.02', 'TIC260647166.02', 'TIC55652896.02', 'TIC55652896.02', 'TI

In [20]:
import pandas as pd
import numpy as np
import panel as pn
import io

pn.extension()


df = all_targets_df

# Get the column names
column_names = df.columns.tolist()
print(column_names)

relevant_columns = ['date_obs', 'target', 'tic_name', 'period', 'depth', 'band', 'quality', 'detection ', 'Obs_Outcome', 'TFOP status', 'notes', 'radius', 'vmag']

# Keep only the named columns in the list
df = df[relevant_columns]

# Sanitize all the columns

# Convert all depth values to string
df['depth'] = df['depth'].astype(str)
# Remove all trailing spaces from the column depth
df['depth'] = df['depth'].str.strip()
# Convert all commas in depth to dots and convert to float
df['depth'] = df['depth'].str.replace(',', '.')
# Convert all depth values to float
df['depth'] = df['depth'].astype(float)
# Remove trailing spaces
df.columns = df.columns.str.strip()

# Convert the columns to the right type
df['date_obs'] = pd.to_datetime(df['date_obs'])
df['period'] = pd.to_numeric(df['period'])
df['depth'] = pd.to_numeric(df['depth'])


# Get the minimum value of period
df['period'].min()

# Configuration dictionary
config = {
    'period': {'min': df['period'].min(), 'max': 250, 'step': 3}, # (df['period'].max() - df['period'].min()) / 10
    'band': {'options': df['band'].unique().tolist()},
    'depth': {'min': df['depth'].min(), 'max': df['depth'].max(), 'step': 0.5},
}

# Function to create slider and input widgets
def create_slider_input(col, config):
    slider = pn.widgets.RangeSlider(
        name=col,
        start=config['min'],
        end=config['max'],
        step=config['step'],
        value=(config['min'], config['max'])
    )
    input_low = pn.widgets.FloatInput(name=f"{col}_low", value=config['min'])
    input_high = pn.widgets.FloatInput(name=f"{col}_high", value=config['max'])

    # Set up callbacks to update slider and input widgets
    def update_slider(event):
        slider.value = (input_low.value, input_high.value)

    def update_inputs(event):
        input_low.value, input_high.value = slider.value

    slider.param.watch(update_inputs, 'value')
    input_low.param.watch(update_slider, 'value')
    input_high.param.watch(update_slider, 'value')
    
    return slider, input_low, input_high

# Create slider, input and select widgets
slider_dict = {}
input_dict = {}
select_dict = {}

for col, conf in config.items():
    if 'min' in conf and 'max' in conf and 'step' in conf:
        slider, input_low, input_high = create_slider_input(col, conf)
        slider_dict[col] = slider
        input_dict[f"{col}_low"] = input_low
        input_dict[f"{col}_high"] = input_high
    elif 'options' in conf:
        options = ['all'] + conf['options']  # add 'all' as the default option
        select = pn.widgets.Select(name=col, options=options, value='all')  # set 'all' as the default value
        select_dict[col] = select

# Define a global variable to store the filtered DataFrame
global_filtered_df = None

# Function to filter dataframe
def filter_dataframe(**values):
    mask = np.ones(len(df), dtype=bool)

    # filtering for dropdown menu columns
    for col, widget in select_dict.items():
        if widget.value != 'all':  # only filter if the selected value is not 'all'
            mask &= df[col] == widget.value
        if col in values:  # removing this from the dictionary so it isn't processed as a slider/input widget
            del values[col]
    
    # filtering for slider/input widgets
    for col in values:
        if col.endswith("_low") or col.endswith("_high"):
            col = col.rsplit("_", 1)[0]
            mask &= df[col].between(values[f"{col}_low"], values[f"{col}_high"])
    return df[mask]

# Update interactive display function to save filtered df in the global variable
@pn.depends(**slider_dict, **input_dict, **select_dict)
def display_dataframe(**values):
    global global_filtered_df
    global_filtered_df = filter_dataframe(**values)
    return global_filtered_df

# Update download functionality to use the global variable
def download_csv():
    csv = global_filtered_df.to_csv(index=False)
    return io.BytesIO(csv.encode())

download_button = pn.widgets.FileDownload(callback=download_csv, filename='filtered_data.csv')

# No changes here
widgets = pn.Column(*(list(slider_dict.values()) + list(input_dict.values()) + list(select_dict.values()) + [download_button]))
app = pn.Row(widgets, display_dataframe)


['date_obs', 'target', 'tic_name', 'period', 'depth', 'BJD start', 'BJD end', 'band', 'quality', 'detection ', 'Obs_Outcome', 'TFOP status', 'other', 'notes', 'Unnamed: 14', 'vmag', 'duration', 'radius']


/tmp/ipykernel_12596/276967908.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['depth'] = df['depth'].astype(str)
/tmp/ipykernel_12596/276967908.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['depth'] = df['depth'].str.strip()
/tmp/ipykernel_12596/276967908.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [21]:
app.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'3717d1ba-ab5d-4bd6-8f02-f91dfe0fd354': {'version…